In [2]:
import numpy as np


## Codigo LU

In [3]:
def LU_factorizacion(A):
    n = A.shape[0]  # Número de filas (o columnas) de la matriz A
    U = A.copy()  # Crear una copia de A para U
    L = np.eye(n)  # Inicializar L como una matriz identidad de nxn

    # Proceso de factorización LU
    for k in range(n - 1):  # Iterar sobre cada columna excepto la última
        for i in range(k + 1, n):  # Iterar sobre cada fila por debajo del pivote actual
            L[i, k] = U[i, k] / U[k, k]  # Calcular el multiplicador y asignarlo a L
            U[i, k:] = U[i, k:] - L[i, k] * U[k, k:]  # Actualizar la fila i de U restando el múltiplo de la fila k

    return L, U  # Devolver las matrices L y U


def resolver_LU(L, U, b):
    n = b.shape[0]

    y = np.zeros_like(b)
    for i in range(n):
        y[i] = b[i] - np.dot(L[i, :i], y[:i])

    x = np.zeros_like(b)
    for i in range(n - 1, -1, -1):
        x[i] = (y[i] - np.dot(U[i, i + 1:], x[i + 1:])) / U[i, i]

    return x

## Codigo LU Factorización parcial

In [37]:
def LU_factorizacion_parcial(A):
    n = A.shape[0]
    U = A.astype(float).copy()  # Asegurarse de que U sea de tipo float
    L = np.eye(n, dtype=float)  # L como matriz identidad de tipo float
    P = np.eye(n, dtype=float)  # P como matriz identidad de tipo float

    for k in range(n):
        # Pivoteo parcial: encuentra el índice del mayor valor absoluto en la columna k
        max_index = np.argmax(np.abs(U[k:n, k])) + k

        # Intercambiar filas en U y en P
        if k != max_index:
            U[[k, max_index]] = U[[max_index, k]]
            P[[k, max_index]] = P[[max_index, k]]
            if k > 0:
                L[[k, max_index], :k] = L[[max_index, k], :k]

        # Eliminación gaussiana
        for i in range(k + 1, n):
            L[i, k] = U[i, k] / U[k, k]
            U[i, k:] -= L[i, k] * U[k, k:]

    return P, L, U


def resolver_LU_Parcial(P, L, U, b):

    # Resuelve el sistema Ax = b usando PA = LU
    # Paso 1: Pb = b (aplicar P)
    Pb = np.dot(P, b)

    # Paso 2: Resolver Ly = Pb
    y = np.zeros_like(Pb, dtype=float)
    for i in range(len(b)):
        y[i] = Pb[i] - np.dot(L[i, :i], y[:i])

    # Paso 3: Resolver Ux = y
    x = np.zeros_like(y, dtype=float)
    for i in range(len(y) - 1, -1, -1):
        x[i] = (y[i] - np.dot(U[i, i + 1:], x[i + 1:])) / U[i, i]

    return x

## Codigo LU factorización total

In [106]:
def lu_factorization_total(A):
    n = A.shape[0]
    U = A.astype(float).copy()
    L = np.eye(n, dtype=float)
    P = np.eye(n, dtype=float)
    Q = np.eye(n, dtype=float)

    for k in range(n):
        # Pivoteo total: encuentra el índice del mayor valor absoluto
        max_index = np.unravel_index(np.argmax(np.abs(U[k:n, k:n])), U[k:n, k:n].shape)
        max_row, max_col = max_index[0] + k, max_index[1] + k

        # Intercambiar filas en U y P
        if k != max_row:
            U[[k, max_row]] = U[[max_row, k]]
            P[[k, max_row]] = P[[max_row, k]]

        # Intercambiar columnas en U y Q
        if k != max_col:
            U[:, [k, max_col]] = U[:, [max_col, k]]
            Q[[k, max_col]] = Q[[max_col, k]]

        # Eliminación gaussiana
        for i in range(k + 1, n):
            L[i, k] = U[i, k] / U[k, k]
            U[i, k:] -= L[i, k] * U[k, k:]

    return P, L, U, Q

def resolver_LU_Total(P, Q, L, U, b):
    Pb = np.dot(P, b)

    # Paso 2: Resolver Ly = Pb
    y = np.zeros_like(Pb, dtype=float)
    for i in range(len(b)):
        y[i] = Pb[i] - np.dot(L[i, :i], y[:i])

    # Paso 3: Resolver Ux = y
    x = np.zeros_like(y, dtype=float)
    for i in range(len(y) - 1, -1, -1):
        x[i] = (y[i] - np.dot(U[i, i + 1:], x[i + 1:])) / U[i, i]

    # Paso 4: Aplicar Q a x
    x = np.dot(Q, x)

    return x

## a)

In [107]:
A1=np.array([[4,-1,3],[-8,4,-7],[12,1,8]])
b1=np.array([[-8],[19],[-19]])

### LU

In [108]:
L11,U11=LU_factorizacion(A1)
L11@U11

array([[ 4., -1.,  3.],
       [-8.,  4., -7.],
       [12.,  1.,  8.]])

In [109]:
resolver_LU(L11,U11,b1)

array([[-1],
       [ 1],
       [-1]])

### LU parcial

In [110]:
P12,L12,U12=LU_factorizacion_parcial(A1)
P12@A1


array([[12.,  1.,  8.],
       [-8.,  4., -7.],
       [ 4., -1.,  3.]])

In [111]:
L12@U12

array([[12.,  1.,  8.],
       [-8.,  4., -7.],
       [ 4., -1.,  3.]])

In [112]:
resolver_LU_Parcial(P12,L12,U12,b1)

array([[-1.],
       [ 1.],
       [-1.]])

### LU total

In [113]:
P13, L13, U13, Q13=lu_factorization_total(A1)
P13@A1@Q13

array([[12.,  1.,  8.],
       [-8.,  4., -7.],
       [ 4., -1.,  3.]])

In [114]:
L13@U13

array([[12.,  1.,  8.],
       [-8.,  4., -7.],
       [ 4., -1.,  3.]])

In [115]:
resolver_LU_Total(P13, Q13, L13, U13, b1)

array([[-1.],
       [ 1.],
       [-1.]])

## b)

In [116]:
A2=np.array([[1,4,-2,1],[-2,-4,-3,1],[1,16,-17,9],[2,4,-9,-3]])
b2=np.array([[3.5],[-2.5],[15],[10.5]])

###LU

In [117]:
L21,U21=LU_factorizacion(A2)
L21@U21

array([[  1.,   4.,  -2.,   1.],
       [ -2.,  -4.,  -3.,   1.],
       [  1.,  16., -17.,   9.],
       [  2.,   4.,  -9.,  -3.]])

In [118]:
resolver_LU(L21,U21,b2)

array([[-0.5],
       [ 1. ],
       [-0.5],
       [-1. ]])

###LU parcial

In [119]:
P22,L22,U22=LU_factorizacion_parcial(A2)
P22@A2

array([[ -2.,  -4.,  -3.,   1.],
       [  1.,  16., -17.,   9.],
       [  2.,   4.,  -9.,  -3.],
       [  1.,   4.,  -2.,   1.]])

In [120]:
L22@U22

array([[ -2.,  -4.,  -3.,   1.],
       [  1.,  16., -17.,   9.],
       [  2.,   4.,  -9.,  -3.],
       [  1.,   4.,  -2.,   1.]])

In [121]:
resolver_LU_Parcial(P22,L22,U22,b2)

array([[-0.5],
       [ 1. ],
       [-0.5],
       [-1. ]])

### LU total

In [134]:
P23, L23, U23, Q23=lu_factorization_total(A2)
P23@A2@Q23

array([[  9., -17.,   1.,  16.],
       [ -3.,  -9.,   2.,   4.],
       [  1.,  -3.,  -2.,  -4.],
       [  1.,  -2.,   1.,   4.]])

In [135]:
L23@U23

array([[-17.        ,   9.        ,  16.        ,   1.        ],
       [ -3.        ,  -6.17647059,  -1.64705882,   1.64705882],
       [ -2.        ,   1.        ,  -4.63636364,  -2.15909091],
       [ -9.        ,   4.17647059,  10.28342246,   1.51203209]])

In [124]:
resolver_LU_Total(P23, Q23, L23, U23, b2)

array([[ 21.02274816],
       [-57.69761029],
       [-24.04294786],
       [  2.78116644]])

## c)

In [125]:
A3=np.array([[4,5,-1,2,3],[12,13,0,10,3],[-8,-8,5,-11,4],[16,18,-7,20,4],[-4,-9,31,-31,-1]])
b3=np.array([[34],[93],[-33],[131],[-58]])

###LU

In [127]:
L31,U31=LU_factorizacion(A3)
L31@U31

array([[  4.,   5.,  -1.,   2.,   3.],
       [ 12.,  13.,   0.,  10.,   3.],
       [ -8.,  -8.,   5., -11.,   4.],
       [ 16.,  18.,  -7.,  20.,   4.],
       [ -4.,  -9.,  31., -31.,  -1.]])

In [128]:
resolver_LU(L31,U31,b3)

array([[1],
       [2],
       [3],
       [4],
       [5]])

###LU parcial

In [130]:
P32,L32,U32=LU_factorizacion_parcial(A3)
P32@A3

array([[ 16.,  18.,  -7.,  20.,   4.],
       [ -4.,  -9.,  31., -31.,  -1.],
       [ -8.,  -8.,   5., -11.,   4.],
       [  4.,   5.,  -1.,   2.,   3.],
       [ 12.,  13.,   0.,  10.,   3.]])

In [131]:
L32@U32

array([[ 16.,  18.,  -7.,  20.,   4.],
       [ -4.,  -9.,  31., -31.,  -1.],
       [ -8.,  -8.,   5., -11.,   4.],
       [  4.,   5.,  -1.,   2.,   3.],
       [ 12.,  13.,   0.,  10.,   3.]])

In [133]:
resolver_LU_Parcial(P32,L32,U32,b3)

array([[1.],
       [2.],
       [3.],
       [4.],
       [5.]])

###LU total

In [137]:
P33,L33,U33,Q33=lu_factorization_total(A3)
P33@A3@Q33

array([[ -1.,  -9.,  -4., -31.,  31.],
       [  4.,  18.,  16.,  20.,  -7.],
       [  4.,  -8.,  -8., -11.,   5.],
       [  3.,   5.,   4.,   2.,  -1.],
       [  3.,  13.,  12.,  10.,   0.]])

In [139]:
L33@U33

array([[ 31.        ,  -9.        ,  -1.        , -31.        ,
         -4.        ],
       [  0.        ,  15.96774194,   3.77419355,  13.        ,
         15.09677419],
       [  5.        ,  -8.        ,   4.        , -11.        ,
         -8.        ],
       [ -7.        ,  15.03225806,   3.22580645,  14.97523001,
         13.00513663],
       [ -1.        ,   5.        ,   3.        ,   4.02476999,
          3.89808917]])

In [140]:
resolver_LU_Total(P33,Q33,L33,U33,b3)

array([[ -37.44236411],
       [ 201.29899465],
       [-238.98985246],
       [  51.23078214],
       [  75.75623949]])